# ゲームの基本を理解しよう

## pygameのインストール
まずはpygameをインストールしよう。       
↓を実行しよう。

In [ ]:
!pip install pygame

***
## 実例を見てみよう
pygameの中身に入る前に、ゲームってどのように表示されているのか基本を押さえます。  
まずは下記のブロック崩しのコードを実行してみましょう。  

sキーでスタート   
方向キーでバーの操作

In [ ]:
# ブロック崩しのデモ

#***************
# ライブラリのインポート
#***************
import pygame as pg                             # pygame全体
from pygame.locals import *                     # 内部変数などにアクセスできる
import sys                                      # 

#***************
# 初期化
#***************
pg.init()                                       # 初期化
clock = pg.time.Clock()                         # 時間設定

#***************
# 画面設定
#***************
screen = pg.display.set_mode((800,600))         # 画面設定
screen_rect = screen.get_rect()                 # 画面の位置情報
pg.display.set_caption('Hello World')           # windownの左上のテキスト

#***************
# 登場人物の設定
#***************
# バー
bar = pg.Surface((100,10))                      # 領域を確保
bar.set_colorkey((0,0,0))                       # 透過色の設定
pg.draw.rect(bar,(255,255,255),(0,0,100,10))    # 領域内を描画する
bar_rect = bar.get_rect()                       # barのRect(位置情報)の取得      
bar_rect.topleft=(screen_rect.centerx-50,screen_rect.bottom-50)      # barの左上位置を決める

# ボール
circ = pg.Surface((20,20))                      # 領域を確保
circ.set_colorkey((0,0,0))                      # 透過色の設定
pg.draw.circle(circ,(255,255,255),(10,10),10)   # 領域内を描画する
circ_rect = circ.get_rect()                     # circのRect(位置情報)の取得
circ_rect.topleft=(screen_rect.centerx-10,screen_rect.bottom-70)      # ++ circの左上位置を決める

# ブロック
block       = {}                                # 領域
block_rect  = {}                                # Rect(位置情報)
target      = {}                                # ブロックの表示の有無を指定する変数

xcnt=0                                          # x方向
ycnt=0                                          # y方向
for i in range(30):
    block[i] = pg.Surface((50,25))                      # 領域を確保
    block[i].set_colorkey((0,0,0))                      # 透過色の設定
    pg.draw.rect(block[i],(255,255,255),(0,0,50,25))    # 領域内を描画する
    block_rect[i] = block[i].get_rect()                 # blockのRect(位置情報)の取得       
    block_rect[i].topleft=(100+(xcnt*60),50+(ycnt*60))  # blockの左上位置を決める
    target[i] = True                                    # ブロックはすべて表示するため、全部Trueとする

    # 加算値の計算
    if xcnt == 9 :                                      # x方向に9まで数えると、yを1つ数える
        ycnt += 1 

    if xcnt / 9 == 1:                                   # x方向に9まで数えると、カウントを0にする
        xcnt = 0
    else:                                               # x方向に9まで数える
        xcnt += 1

#***************
# ゲーム内容
#***************
dx = 5 
dy = 5
status = "IDLE"                                 # ステータス
while True:
    pressd_keys = pg.key.get_pressed()          # キー操作の取得
    if pressd_keys[K_s]:
        status = "START" # ++ スタート

    if not(True in target.values()):
        status = "END" # ++ エンド
            
    #---------------
    # 画面の操作
    #---------------
    screen.fill((100,100,100))                  # 画面の色設定

    #---------------
    # 登場人物の操作
    #---------------

    if status == "START" :                      # ++ スタートすると登場人物の操作を開始
        #-- バーの操作
        if pressd_keys[K_LEFT]:                     # 左キーの操作
            bar_rect.move_ip(-10,0)
        if pressd_keys[K_RIGHT]:                    # 右キーの操作
            bar_rect.move_ip(10,0)
        if pressd_keys[K_UP]:                       # 上キーの操作
            bar_rect.move_ip(0,-10)
        if pressd_keys[K_DOWN]:                     # 下キーの操作
            bar_rect.move_ip(0,10)   

        #-- ボールの操作
        circ_rect.move_ip(dx,dy)                    # ボールを動かす 

        #-- バーとボールの衝突判定
        if bar_rect.colliderect(circ_rect):
            if circ_rect.left < bar_rect.left or \
                circ_rect.right > bar_rect.right:   # ボールが左右から衝突した場合の判定。
                dx=-dx
            if circ_rect.top < bar_rect.top or \
                circ_rect.bottom > bar_rect.bottom: # ボールが上下から衝突した場合の判定。
                dy=-dy    
        bar_rect.clamp_ip(screen_rect)             # 画面外に出ないように戻す操作 


        #-- ボールの衝突判定
        if circ_rect.left < screen_rect.left or \
           circ_rect.right > screen_rect.right:     # ボールが左右から出そうになると動きを反転する。
            dx=-dx
        if circ_rect.top < screen_rect.top or \
          circ_rect.bottom > screen_rect.bottom:    # ボールが上下から出そうになると動きを反転する。
            dy=-dy  
        circ_rect.clamp_ip(screen_rect)             # 画面外に出ないように戻す操作   


        #-- ++ ロックとボールの衝突判定
        for i in target:
            if target[i]:                           # ++ targetがTrueのときだけ衝突判定する
                if block_rect[i].colliderect(circ_rect):
                    if circ_rect.left < block_rect[i].left or circ_rect.right > block_rect[i].right:
                        dx=-dx
                    if circ_rect.top < block_rect[i].top or circ_rect.bottom > block_rect[i].bottom:
                        dy=-dy
                    target[i] = False
                    break
    
    #---------------
    # 画面の表示
    #---------------    
    screen.blit(circ,circ_rect)                  # ボールを配置する
    screen.blit(bar,bar_rect)                    # バーを配置する

    for i in target:                             # ブロックを配置する
        if target[i]:
            screen.blit(block[i],block_rect[i])
    pg.display.update()                          # 画面表示

    #---------------
    # 終了判定
    #---------------
    # バツボタンを押すとゲームが終了するという処理
    for event in pg.event.get():
        if event.type == QUIT:
            pg.quit()
            sys.exit()

    if status == "END" :# ++ エンド
            pg.quit()
            sys.exit()

    clock.tick(60)                                  #FPS設定 

***
## 舞台と登場人物
### オブジェクト
ゲーム画面には、「画面」「ブロック」「ボール」「バー」という舞台と登場人物たちが存在します。  
(「画面」が舞台で、「ブロック」「ボール」「バー」が登場人物のイメージ)  
これらをオブジェクトと呼びます。このオブジェクトという言葉は重要！！ゲーム以外でもよく登場します。


![ゲーム画面](./image/png/game_object.png)

### 登場人物の移動
舞登場人物たちは、どうやって移動しているか？  
答えはパラパラ漫画のイメージで、ちょっとずつ移動しては、何回も表示するということを繰り返している。   
これであたかもボールは動いているように見えています。    



### 位置情報
ゲームで登場人物を動かすときには、位置情報が大変重要。  

画面は左上を(0,0)としたX方向,Y方向の座標で表されます。  
例えば、ボールは最初(400,500)の位置にいて、ゲームがスタートすると、  
(401,501)→(402,502)→(403,503)→(404,504)と変化していき、  
その都度画面を表示しなおしているので右上に動き出しているように見えます。

画面の1秒間に表示する回数をFPS(frames per second)と呼びます。  
これが大きいと滑らかに動いているように見えて、小さいとコマ送りのようなカクカク動作に見えます。

![位置情報](./image/png/game_object_rect.png)

### 衝突判定
登場人物が位置情報を持っていることは分かったとして、     
その登場人物同士がぶつかったように動くのはなぜなのか？  

やはりこれも位置情報を使っている。  

例えば、ボールがブロックに触れると反射するのは、  
ブロックとボールの位置が重なることを判定して、    
ボールの動きを変化させるようにコードを書くことで、  
あたかも反射したかのような動作を作ることができる。

![位置情報](./image/png/game_conflict.png)

以上の基本的な考え方を踏まえて、   
実際に順番にコードを見ていきましょう！！  
pygame_ブロック崩し.ipynbに進んでください。